In [ ]:
import numpy as np


In [ ]:
import pandas as pd


In [ ]:
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpProblem, LpVariable

In [ ]:
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpProblem, LpVariable

In [ ]:
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpProblem, LpVariable

In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pulp import LpMaximize, LpMinimize, LpStatus, lpSum, LpProblem, LpVariable

In [ ]:
import copy

In [ ]:
def initiation_tb(goal_vars, goal_q, goal_dir, cst_a, cst_b, cst_signs):
    upper_idx = np.arange(goal_vars.shape[0])
    left_idx = np.arange(upper_idx[-1] + 1, upper_idx[-1] + 1 + cst_b.shape[0])

    if (goal_dir == 'max'):
        goal_vars *= -1
        goal_q *= -1

    for i in range(cst_a.shape[0]):
        if cst_signs[i] == '>' or cst_signs[i] == '>=':
            cst_a[i] *= -1
            cst_b[i] *= -1  
    return cst_a, cst_b, goal_vars, upper_idx, left_idx, -goal_q

In [ ]:
def new_tb(a, b, c, seld_row, seld_column, cr_f, inds_basis_vars, inds_free_vars):
    antic_a = copy.deepcopy(a)
    antic_c = copy.deepcopy(c)
    antic_b = copy.deepcopy(b)
    seld_a = a[seld_row, seld_column]
    a[seld_row] /= seld_a
    b[seld_row] /= seld_a
    c[seld_column] /= -seld_a
    a[:, seld_column] = [
        antic_a[i, seld_column] / -seld_a if i != seld_row else antic_a[i, seld_column] for i in
        range(antic_a.shape[0])]
    for i in range(antic_a.shape[0]):
        if (i == seld_row):
            continue
        b[i] -= antic_b[seld_row] * antic_a[i, seld_column] / seld_a
        for j in range(antic_a.shape[1]):
            if (j == seld_column):
                continue
            a[i, j] -= antic_a[seld_row, j] * antic_a[i, seld_column] / seld_a
    for j in range(antic_a.shape[1]):
        if (j == seld_column):
            continue
        c[j] -= antic_a[seld_row, j] * antic_c[seld_column] / seld_a
    cr_f -= antic_b[seld_row] * antic_c[seld_column] / seld_a
    inds_basis_vars[seld_row], inds_free_vars[seld_column] = inds_free_vars[seld_column], inds_basis_vars[
        seld_row]
    return inds_free_vars, inds_basis_vars, b, cr_f

In [ ]:

def return_cc(inds_free_vars, inds_basis_vars, b, cr_f):
    xs = np.zeros((1, inds_free_vars.shape[0] + inds_basis_vars.shape[0]))[0]
    for ind_bas, cr_b in zip(inds_basis_vars, b): xs[ind_bas] = cr_b
    return xs, -cr_f




In [ ]:
def check_optim(c):
    is_check = (c >= 0).all()
    return is_check

In [ ]:
def sel_column(c):
    ch_column =  np.argmin(c)
    return ch_column

In [ ]:
def sel_row(seld_column, b, a): 
  ch_row =  np.argmin([b[i] / a[i, seld_column] 
                       if a[i, seld_column] > 0 
                       else np.inf for i in range(b.shape[0])])
  return ch_row

In [ ]:
def pront(basis_vars, free_vars, a, b, c, cr_f):
    simp_table = pd.DataFrame(index=basis_vars, columns=free_vars).fillna(0)
    columns = simp_table.columns.values
    for i in range(columns.shape[0]): 
      simp_table[columns[i]] = a[:, i]
    simp_table.loc['c'] = c
    simp_table['b'] = np.append(b, cr_f)
    return simp_table

In [ ]:
def decide(goal_vars, goal_q, goal_dir, cst_a, cst_b, cst_signs):
    iteration_cr = 0
    a, b, c, inds_free_vars, inds_basis_vars, cr_F = initiation_tb(goal_vars, goal_q, goal_dir, cst_a, cst_b,
                                                                 cst_signs)
    while iteration_cr < 10000 and check_optim(c) == False:
        seld_column = sel_column(c)
        seld_row = sel_row(seld_column, b, a)
        new_tb(a, b, c, seld_row, seld_column, cr_F, inds_basis_vars, inds_free_vars)
        iteration_cr += 1
        
    answer = return_cc(inds_free_vars, inds_basis_vars, b, cr_F)
    w_opt = answer[1]
    if (goal_dir == 'max'): w_opt *= -1
    print(pront(inds_basis_vars, inds_free_vars, a, b, c, cr_F))
    print()
    print(f'Повторов: {iteration_cr}')
    print(f'целевые переменные: {answer[0]}')
    print(f'Значение  функции(целевой): {w_opt}')
    return answer[0], w_opt

goal_vars = np.array([3, -2, 4], dtype=np.float32)
goal_q = 22
goal_dir = 'max'
cst_a = np.array([[1, 1, 3],
                  [2, 4, 0],
                  [1, 2, 0]], dtype=np.float32)
cst_b = np.array([12, 14, 6], dtype=np.float32)
signs = ['<=', '<=', '<=', '<=']
res = decide(goal_vars, goal_q, goal_dir, cst_a, cst_b, signs)

          5         1         3     b
2 -0.333333 -0.333333  3.000000   2.0
4 -2.000000  0.000000  0.000000   2.0
0  1.000000  2.000000 -0.000000   6.0
c  1.666667  6.666667  1.333333  22.0

Повторов: 2
целевые переменные: [6. 0. 2. 0. 2. 0.]
Значение  функции(целевой): 22


In [ ]:
# max

c1 = np.array([8, 7])
A_ub = [[4, 3], [2, 1], [2, 3]]
b_ub = [144, 64, 120]

result = decide(-c1, A_ub, b_ub, bounds=(0, np.inf))
print(result)

TypeError: ignored